In [1]:
!pip install pandas influxdb-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.3/746.3 kB 11.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [8]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd


# InfluxDB Connection Details
INFLUXDB_URL = "https://influxdb-battery-demo.apps.sno.pemlab.rdu2.redhat.com/"
INFLUXDB_TOKEN = "admin_token"
INFLUXDB_ORG = "redhat"
INFLUXDB_BUCKET = "bms"

In [9]:
# Initialize Client
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)


def retrieve_battery_data():
    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -1h)
      |> filter(fn: (r) => r["_measurement"] == "battery_data")
    '''
    query_api = client.query_api()
    tables = query_api.query(query, org=INFLUXDB_ORG)

    # Process Results
    data = []
    for table in tables:
        for record in table.records:
            data.append(record.values)

    df = pd.DataFrame(data)
    print(df)  # Print the retrieved data
    return df

In [10]:
df = retrieve_battery_data()

# Drop non-numeric columns
df = df.drop(columns=["result", "table", "_measurement"])

# Save for ML
df.to_csv("./data/battery_raw.csv", index=False)
print("Saved raw battery data from InfluxDB.")

       result  table                           _start  \
0     _result      0 2025-10-06 15:11:50.226605+00:00   
1     _result      0 2025-10-06 15:11:50.226605+00:00   
2     _result      0 2025-10-06 15:11:50.226605+00:00   
3     _result      0 2025-10-06 15:11:50.226605+00:00   
4     _result      0 2025-10-06 15:11:50.226605+00:00   
...       ...    ...                              ...   
9067  _result      8 2025-10-06 15:11:50.226605+00:00   
9068  _result      8 2025-10-06 15:11:50.226605+00:00   
9069  _result      8 2025-10-06 15:11:50.226605+00:00   
9070  _result      8 2025-10-06 15:11:50.226605+00:00   
9071  _result      8 2025-10-06 15:11:50.226605+00:00   

                                _stop                            _time  \
0    2025-10-06 16:11:50.226605+00:00 2025-10-06 15:11:51.638000+00:00   
1    2025-10-06 16:11:50.226605+00:00 2025-10-06 15:11:54.638000+00:00   
2    2025-10-06 16:11:50.226605+00:00 2025-10-06 15:11:57.638000+00:00   
3    2025-10-06 16: